In [1]:
using Pkg
Pkg.add(;path="/Users/az/Documents/GitHub/GLOM_RV_Example")

   Updating git-repo `/Users/az/Documents/GitHub/GLOM_RV_Example`


   Updating registry at `~/.julia/registries/General`
######################################################################### 100.0%
  Resolving package versions...
Updating `~/Documents/GitHub/GLOM_RV_Example/examples/Project.toml`
  [3d369fb6] ~ GLOM_RV_Example v0.1.0 `/Users/az/Documents/GitHub/GLOM_RV_Example#main` ⇒ v0.1.0 `/Users/az/Documents/GitHub/GLOM_RV_Example#main`
Updating `~/Documents/GitHub/GLOM_RV_Example/examples/Manifest.toml`
  [3d369fb6] ~ GLOM_RV_Example v0.1.0 `/Users/az/Documents/GitHub/GLOM_RV_Example#main` ⇒ v0.1.0 `/Users/az/Documents/GitHub/GLOM_RV_Example#main`


In [2]:

Pkg.activate("examples")
Pkg.instantiate()

 Activating environment at `~/Documents/GitHub/GLOM_RV_Example/examples/examples/Project.toml`


In [3]:
import Pkg; Pkg.add("CSV")

using Statistics

# for det()
using LinearAlgebra

# for importing the data from CSV
using DataFrames
using CSV


  Resolving package versions...
No Changes to `~/Documents/GitHub/GLOM_RV_Example/examples/examples/Project.toml`
No Changes to `~/Documents/GitHub/GLOM_RV_Example/examples/examples/Manifest.toml`


In [4]:
# For GLOM
# import Pkg; Pkg.add("GPLinearODEMaker")
import GPLinearODEMaker; GLOM = GPLinearODEMaker

GPLinearODEMaker

In [5]:
# For this module
using GLOM_RV_Example; GLOM_RV = GLOM_RV_Example

GLOM_RV_Example

In [6]:
# For units in orbit fitting functions
import Pkg; Pkg.add("Unitful")
using UnitfulAstro, Unitful

  Resolving package versions...
No Changes to `~/Documents/GitHub/GLOM_RV_Example/examples/examples/Project.toml`
No Changes to `~/Documents/GitHub/GLOM_RV_Example/examples/examples/Manifest.toml`


In [7]:

## Problem setup

# CHANGE: choose a kernel, I suggest 3 for Matern 5/2 or 4 for Quasi-periodic
# kernel
kernel_choice = 4
kernel_names = ["pp", "se", "m52", "qp", "m52_m52", "se_se"]
kernel_name = kernel_names[kernel_choice]
kernel_function, num_kernel_hyperparameters = GLOM.include_kernel(kernel_name)

(GPLinearODEMaker.qp_kernel, 3)

In [8]:
# CHANGE: the stars rotation rate which is used as the first guess for some GLOM
# hyperparameters and starting point for priors
star_rot_rate = 16.28  # days

# importing Yale's 101501 data
data = CSV.read("101501_activity.csv", DataFrame)

# CHANGE: observation times go here
obs_xs = collect(data[!, "Time [MJD]"])
# taking out the mean observation times makes the optimization easier for models
# with periodic parameters
GLOM_RV.remove_mean!(obs_xs)

44-element Array{Float64,1}:
 -269.2213392585909
 -269.2079564661908
 -269.19584069718985
 -269.19022286658947
 -269.1841134480892
 -234.42786105518826
 -234.42343912379147
 -234.41909104769002
 -193.37810670998806
 -187.37832593588973
 -185.377775409288
 -184.34866100998988
 -177.44040562718874
    ⋮
  234.46323363490956
  234.4661339899103
  234.4693607798108
  236.49587611011157
  236.50828411771
  236.5199178333096
  237.46599602491187
  237.47044131001167
  237.47468347791073
  239.4646463973113
  239.46659213601015
  239.4703088426104

In [9]:
# CHANGE: rvs and their errors go here
obs_rvs = data[!, "CCF RV [m/s]"]
inject_ks = GLOM_RV.kep_signal(; K=50u"m/s", P=sqrt(2)*5u"d", M0=rand()*2*π, ω_or_k=rand()*2*π, e_or_h=0.1)
obs_rvs[:] .+= ustrip.(inject_ks.(obs_xs.*u"d"))
obs_rvs_err = data[!, "CCF RV Error [m/s]"]

44-element Array{Float64,1}:
 0.26936
 0.82565
 0.25805
 0.26118
 0.25812
 0.34265
 0.3395
 0.35351
 0.26193
 0.21608
 0.21339
 0.30804
 0.24111
 ⋮
 0.31966
 0.32052
 0.32098
 0.43579
 0.3187
 0.32022
 0.32468
 0.33428
 0.33553
 0.31059
 0.31428
 0.31377

In [10]:
# CHANGE: activity indicators and thier errors go here
# you can actually have as many as you want, but obviously it will take longer
# to fit
obs_indicator1 = data[!, "CCF FWHM [m/s]"]
obs_indicator1_err = data[!, "CCF FWHM Err. [m/s]"]
obs_indicator2 = data[!, "BIS [m/s]"]
obs_indicator2_err = repeat([std(obs_indicator2)], length(obs_indicator2))  # I just put something here


44-element Array{Float64,1}:
 10.252322870488586
 10.252322870488586
 10.252322870488586
 10.252322870488586
 10.252322870488586
 10.252322870488586
 10.252322870488586
 10.252322870488586
 10.252322870488586
 10.252322870488586
 10.252322870488586
 10.252322870488586
 10.252322870488586
  ⋮
 10.252322870488586
 10.252322870488586
 10.252322870488586
 10.252322870488586
 10.252322870488586
 10.252322870488586
 10.252322870488586
 10.252322870488586
 10.252322870488586
 10.252322870488586
 10.252322870488586
 10.252322870488586

In [11]:
# removing means as the GP model assumes zero mean
GLOM_RV.remove_mean!(obs_rvs)
GLOM_RV.remove_mean!(obs_indicator1)
GLOM_RV.remove_mean!(obs_indicator2)

44-element Array{Float64,1}:
  -8.422655705644182
  -4.754837654014075
  -7.332458207144278
  -7.434779861995274
  -6.923625532782985
 -15.087346115699482
 -13.269319910469385
 -13.864474880708876
  26.653436251744424
  -9.172527194707676
  -5.764498492557074
  -1.7898246517967848
  23.605814906886117
   ⋮
  -4.929626857433185
  -3.8037417140264793
  -3.9819511485553747
  -0.5186597054477815
   0.11694401814592226
   0.18717887620672258
  -7.97642490141758
  -7.915982666037685
  -9.576468440242685
  -7.6600565301120795
  -6.019500402399579
  -6.6842541475968815

In [12]:
# CHANGE: change these lines if you add more than 2 indicators
# this takes the data and riffles it together so it takes the form
# [rv_1, ind1_1, ind2_1, rv_2, ind1_2, ind2_2, ...]
n_out = 3  # number of indicators + 1
obs_ys = collect(Iterators.flatten(zip(obs_rvs, obs_indicator1, obs_indicator2)))
obs_noise = collect(Iterators.flatten(zip(obs_rvs_err, obs_indicator1_err, obs_indicator2_err)))


132-element Array{Float64,1}:
  0.26936
 41.0664677798099
 10.252322870488586
  0.82565
 42.5939468151545
 10.252322870488586
  0.25805
 40.951629354073
 10.252322870488586
  0.26118
 40.970265403941
 10.252322870488586
  0.25812
  ⋮
  0.33553
 40.5159586937557
 10.252322870488586
  0.31059
 40.7087102555426
 10.252322870488586
  0.31428
 40.5911287592448
 10.252322870488586
  0.31377
 40.6561879107426
 10.252322870488586

In [13]:
# How many differention orders we want in the GLOM model
n_dif = 2 + 1

3

In [14]:
# CHANGE: consider changing a0 (the GLOM coefficients that are used, see
# commented lines below)
# If all a's active:
problem_definition = GLOM.GLO(kernel_function, num_kernel_hyperparameters, n_dif, n_out, obs_xs, copy(obs_ys); noise=copy(obs_noise), a0=[[1. 1 1];[1 1 1];[1 1 1]])
# problem_definition = GLOM.GLO(kernel_function, num_kernel_hyperparameters, n_dif, n_out, obs_xs, copy(obs_ys); noise=copy(obs_noise), a0=[[1. 1 0];[1 0 1];[1 0 1]])

# Makes the std of each output equal to 1, improves fitting stability
# the normalizations are stored in problem_definition.normals
GLOM.normalize_problem_definition!(problem_definition)

In [15]:
# CHANGE: Setting initial fit values
initial_total_hyperparameters = collect(Iterators.flatten(problem_definition.a0))
initial_hypers = [[star_rot_rate], [star_rot_rate], [star_rot_rate], [star_rot_rate, 2 * star_rot_rate, 1], [star_rot_rate, 2 * star_rot_rate, 1], [star_rot_rate, 2 * star_rot_rate, 1]]
append!(initial_total_hyperparameters, initial_hypers[kernel_choice])

12-element Array{Float64,1}:
  1.0
  1.0
  1.0
  1.0
  1.0
  1.0
  1.0
  1.0
  1.0
 16.28
 32.56
  1.0

In [16]:
## Fitting GLOM Model

# CHANGE: Setting kernel hyperparameter priors and kick function
# kick functions help avoid saddle points
tighten_lengthscale_priors = 3
if kernel_name in ["pp", "se", "m52"]
    kernel_hyper_priors(hps::Vector{<:Real}, d::Integer) =
        GLOM_RV.kernel_hyper_priors_1λ(hps, d, star_rot_rate, star_rot_rate / 2 / tighten_lengthscale_priors)
    add_kick!(hps::Vector{<:Real}) = GLOM_RV.add_kick_1λ!(hps)
elseif kernel_name == "qp"
    kernel_hyper_priors(hps::Vector{<:Real}, d::Integer) =
        GLOM_RV.kernel_hyper_priors_qp(hps, d, [star_rot_rate, 2 * star_rot_rate, 1], [star_rot_rate / 2, star_rot_rate / 2, 0.4] ./ tighten_lengthscale_priors)
    add_kick!(hps::Vector{<:Real}) = GLOM_RV.add_kick_qp!(hps)
elseif kernel_name in ["se_se", "m52_m52"]
    kernel_hyper_priors(hps::Vector{<:Real}, d::Integer) =
        GLOM_RV.kernel_hyper_priors_2λ(hps, d, [star_rot_rate, 2 * star_rot_rate, 1], [star_rot_rate / 2, star_rot_rate / 2, 1] ./ tighten_lengthscale_priors)
    add_kick!(hps::Vector{<:Real}) = GLOM_RV.add_kick_2λ!(hps)
else
    # kernel_hyper_priors(hps::Vector{<:Real}, d::Integer) = custom function
end




add_kick! (generic function with 1 method)

In [17]:
fit1_total_hyperparameters, result = GLOM_RV.fit_GLOM(problem_definition, initial_total_hyperparameters, kernel_hyper_priors, add_kick!)
# fit_GLOM returns a vector of num_kernel_hyperparameters gp hyperparameters
# followed by the GLOM coefficients and the Optim result object
workspace = GLOM.nlogL_matrix_workspace(problem_definition, fit1_total_hyperparameters)

gridsearching over hp10
starting at hp = 16.28 -> 153.15441929584313
searching from 12.21 to 20.35
ending at hp   = 13.838 -> 140.32325156736857
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 13.838, 32.56, 1.0]

[1.6954816435816729, 0.9184294003322562, 0.6696321930555993, 0.9599271067288851, 1.0051392301677462, 1.0177038581184514, 0.41990412421382295, 1.0565877148217702, 1.2352438909913335, 13.892706174355608, 32.554812254548594, 0.9778596124329865]

Iteration:              1
Time so far:            4.457962989807129 s
Unnormalized posterior: 110.96703000820213
Gradient norm:          11.077281715424334

[1.697672799704701, 0.9217616311777097, 0.6362382639636471, 0.962330248281525, 0.9447696046352565, 0.8537928300660877, 0.29278956093671005, 1.0261724302698094, 1.2788853257211437, 13.977355492533043, 32.51588751711165, 1.034286122138493]

Iteration:              2
Time so far:            8.872979879379272 s
Unnormalized posterior: 108.07083764087999
Gradient norm:          5.5677177186

GPLinearODEMaker.nlogL_matrix_workspace{Float64}([-0.3578817166613505, -0.2510974627958365, 0.9201226529872824, 0.25869711647238947, 0.5292853185858168, 0.952854089041436, -3.0950199904743787, -0.5764790316368275, 1.4109296480423346, 14.957831267767002, 31.396034700930613, 0.9374968285456486], Cholesky{Float64,Array{Float64,2}}([0.8689754922698235 0.15762081158881075 … -5.467120148271334e-59 -4.726795870398774e-58; 0.13696862234235593 1.6730292530464412 … 2.4818922889316676e-59 -1.597331038360523e-60; … ; -4.750793422142354e-59 3.2905464874690034e-59 … 1.6389842407309554 0.0010035735607718612; -4.107469768338744e-58 -7.717652168466559e-59 … -0.09592886567510009 1.0787173022634806], 'U', 0), [-0.3578817166613505, -0.2510974627958365, 0.9201226529872824, 0.25869711647238947, 0.5292853185858168, 0.952854089041436, -3.0950199904743787, -0.5764790316368275, 1.4109296480423346, 14.957831267767002, 31.396034700930613, 0.9374968285456486], [[5.255589403297679 -2.83781383432469 … 3.146708599813

In [18]:
## Plotting initial results

plot_xs = collect(LinRange(obs_xs[1]-10, obs_xs[end]+10, 300))
post, post_err, post_obs = GLOM_RV.GLOM_posteriors(problem_definition, plot_xs, fit1_total_hyperparameters)
GLOM_rvs_at_plot_xs, GLOM_ind1_at_plot_xs, GLOM_ind2_at_plot_xs = post
GLOM_rvs_err_at_plot_xs, GLOM_ind1_err_at_plot_xs, GLOM_ind2_err_at_plot_xs = post_err
GLOM_rvs_at_obs_xs, GLOM_ind1_at_obs_xs, GLOM_ind2_at_obs_xs = post_obs

Array dimensions do not match. The matrix can't be symmetric


3-element Array{Array{Float64,1},1}:
 [31.75034859422527, 31.946487301185424, 32.118832334897995, 32.197045569978, 32.28087254971121, 15.43241114095533, 15.481897395505646, 15.530376152551545, -42.46615635503414, -47.36284568628137  …  42.036836406252206, -47.96534506042284, -48.07683815758778, -48.17857007812205, -48.12108889782577, -48.07077618214829, -48.02214725860125, 19.928624243254184, 19.967496797284298, 20.041199439712926]
 [5.218331197623229, 5.205949091759689, 5.1940440291382535, 5.188300070929596, 5.1818927512698245, 5.7473421761178205, 5.753122464388962, 5.758767053897681, -4.047156609588896, -8.102173818905307  …  -8.396779466840211, -11.438059557163143, -11.359727605649578, -11.285909349863868, -4.558370665117524, -4.524005455225578, -4.491184719254785, 7.6361447550996, 7.6366049136350815, 7.637399790372992]
 [-6.209573632333951, -6.264053078959602, -6.313257203960036, -6.336031433210499, -6.3607677001145495, -9.842704407262643, -9.850634061669167, -9.858430469913921, 15

In [19]:
obs_rvs[1]

31.550019949186208